# Tulime network analysis

The data regarding the project was first obtained through an Gsheet compiled partially by an organization component and partially by me, employing the data provided by the association thrugh mails and Whatsapp conversations. <br>
This sheet served, once completed, to extract data regarding single projects and convert them into CSVs in order to later access them through python and perform analysis and operation on data through <a href="https://pandas.pydata.org/">Pandas</a> library.

In [ ]:
import numpy as np
import pandas as pd
import random
import re

In [ ]:
def get_all_in_dir(dir):
    for filename in os.listdir(dir):
        f = os.path.join(dir, filename)
        if os.path.isfile(f) and f[-4:] == ".csv":
            yield f

First of all the random integers for the beneficiaris of Maji in Pomerini, Kitowo and Mwawambala were extracted.<br>
A corresponding number of rows was added to the excel file with "Name*n*" in the "NAME" field and the village name in the corresponding one.

In [ ]:
rand_kitowo=random.randint(2500, 3000)
rand_pomerini=random.randint(850, 900)
rand_mwawambala=random.randint(850, 900)

df_maji = pd.DataFrame(index=np.arange(rand_kitowo+rand_pomerini+rand_mwawambala), columns=["NAME", "VILLAGE", "AGE", "SEX", "FAMILY", "START"])
idx = df_maji.index.values.astype(int)
idx = idx [1:]
for el in idx:
    df_maji.loc[el, "NAME"] = "Name"+str(el)
    if el in range(1, rand_kitowo):
        df_maji.loc[el, "VILLAGE"] = "Kitowo"
    elif el in range(rand_kitowo, rand_kitowo+rand_pomerini):
        df_maji.loc[el, "VILLAGE"] = "Pomerini"
    elif el in range(rand_kitowo+rand_pomerini, rand_kitowo+rand_pomerini+rand_mwawambala):
        df_maji.loc[el, "VILLAGE"] = "Mawambala"
df_maji=df_maji.dropna(thresh=2)
#df_maji.to_csv(r"projects_csv/Maji.csv")


In this first step we need to obtain the distribution of values absent in other CSVs from the that present a more complete amount of information. To do so we first of all extract the data we have and organize it in dictionaries (to conver in CSVs) containing the values of interest and the numbre of peope in realtion to each of the values.

In [ ]:
to_scan= [file for file in get_all_in_dir("projects_csv")]

sex_dict = dict()
age_dict=dict()
age_dict_children = dict()
family_dict=dict()

for file in to_scan:
    df = pd.read_csv(file, encoding="utf-8")
    df=df.fillna("NaN")
    for col in df.columns:
        if col == "SEX":
            for el in df[col]:
                if el in sex_dict.keys():
                    sex_dict[el]+=1
                elif el !="NaN":
                    sex_dict[el]=1

        if col == "AGE" and file not in ["projects_csv\TupoPamoja.csv", "projects_csv\PiantalaSubito.csv"]:
            for el in df[col]:
                if el in age_dict.keys():
                    age_dict[el]+=1
                elif el !="NaN":
                    age_dict[el]=1

        if col== "AGE":
            for el in df[col]:
                if el in age_dict_children.keys():
                    age_dict_children[el]+=1
                elif el !="NaN":
                    age_dict_children[el]=1

        
        if col == "FAMILY":
            for el in df[col]:
                if el in family_dict.keys():
                    family_dict[el]+=1
                elif el !="NaN":
                    family_dict[el]=1

df_sex = pd.DataFrame.from_dict(sex_dict, orient='index')
#df_sex.to_csv(r"projects_csv/measures/sex_count.csv")

age_dict = dict(sorted(age_dict.items(), key=lambda item: item[1], reverse=True))
df_age = pd.DataFrame.from_dict(age_dict, orient='index')
#df_age.to_csv(r"projects_csv/measures/age_count.csv")

age_dict_children = dict(sorted(age_dict_children.items(), key=lambda item: item[1], reverse=True))
df_age_children = pd.DataFrame.from_dict(age_dict_children, orient='index')
#df_age_children.to_csv(r"projects_csv/measures/agetotal_count.csv")

family_dict = dict(sorted(family_dict.items(), key=lambda item: item[1], reverse=True))
df_family = pd.DataFrame.from_dict(family_dict, orient='index')
#df_family.to_csv(r"projects_csv/measures/family_count.csv")

            


Columns were then renaimed as "*VARIABLE*" and "PEOPLE"

# Extracting distributions from the values obtained
In order to fill missing gaps in the other CSVs we need to transform the data already in our possess into distributions of variables.

In [ ]:
sum_sex = 0
sum_age = 0
sum_agechildren = 0
sum_family = 0

for el in sex_dict.keys():
    sum_sex += sex_dict[el]
for el in sex_dict.keys():
    sex_dict[el]=sex_dict[el]/sum_sex

for el in age_dict.keys():
    sum_age += age_dict[el]
for el in age_dict.keys():
    age_dict[el]=age_dict[el]/sum_age

for el in age_dict_children.keys():
    sum_agechildren += age_dict_children[el]
for el in age_dict_children.keys():
    age_dict_children[el]=age_dict_children[el]/sum_agechildren

for el in family_dict.keys():
    sum_family += family_dict[el]
for el in family_dict.keys():
    family_dict[el]=family_dict[el]/sum_family

df_sex = pd.DataFrame.from_dict(sex_dict, orient='index')
#df_sex.to_csv(r"projects_csv/measures/sex_distribution.csv")

age_dict = dict(sorted(age_dict.items(), key=lambda item: item[1], reverse=True))
df_age = pd.DataFrame.from_dict(age_dict, orient='index')
#df_age.to_csv(r"projects_csv/measures/age_distribution.csv")

age_dict_children = dict(sorted(age_dict_children.items(), key=lambda item: item[1], reverse=True))
df_age_children = pd.DataFrame.from_dict(age_dict_children, orient='index')
#df_age_children.to_csv(r"projects_csv/measures/agetotal_distribution.csv")

family_dict = dict(sorted(family_dict.items(), key=lambda item: item[1], reverse=True))
df_family = pd.DataFrame.from_dict(family_dict, orient='index')
#df_family.to_csv(r"projects_csv/measures/family_distribution.csv")


Columns were then renaimed as "*VARIABLE*" and "PERCENTAGE"

# Filling missing values in CSVs using obtained distributions
Now we need to use those values to fill missing cells in the remaining CSVs. WE will care about exceptions such as TupoPamoja and Mwalimu, that have restricted age range and consider the total age CSV just for MAJI, which is a project devoted to water supply: this could seem an uneveness in balance, since children are high in number but one of te interventions of MAJI regards only the school district, that by itself counts 900 inhabitants of younger ages with respect to other villagers. <br>
In order to do so <a href="https://numpy.org/doc/stable/">Numpy</a> was employed.

In [ ]:
to_scan= [file for file in get_all_in_dir("projects_csv")]

df_age = pd.read_csv("projects_csv/measures/age_distribution.csv")
df_age_total = pd.read_csv("projects_csv/measures/agetotal_distribution.csv")
df_sex = pd.read_csv("projects_csv/measures/sex_distribution.csv")
df_family = pd.read_csv("projects_csv/measures/family_distribution.csv")

for file in to_scan:
    df = pd.read_csv(file, encoding="utf-8")
    missing_age = df["AGE"].isnull()
    missing_sex = df["SEX"].isnull()
    missing_family = df["FAMILY"].isnull()
    if file.split("\\")[1] not in ["Maji.csv", "Mwalimu.csv", "TupoPamoja.csv"]:
        df.loc[missing_age, "AGE"] = np.random.choice(df_age["AGE"], size=len(df[missing_age]), p=df_age["PERCENTAGE"])
    elif file.split("\\")[1] not in ["TupoPamoja.csv", "Mwalimu.csv"]:
        df.loc[missing_age, "AGE"] = np.random.choice(df_age_total["AGE"], size=len(df[missing_age]), p=df_age_total["PERCENTAGE"])
    df.loc[missing_sex, "SEX"] = np.random.choice(df_sex["SEX"], size=len(df[missing_sex]), p=df_sex["PERCENTAGE"])
    df.loc[missing_family, "FAMILY"] = np.random.choice(df_family["FAMILY"], size=len(df[missing_family]), p=df_family["PERCENTAGE"])
    if file.split("\\")[1] == "TupoPamoja.csv":
        df.loc[missing_age, "AGE"] = np.random.choice(range(18, 49, 6), size=len(df[missing_age]))
        for row in df.iterrows():
            age = row[1]["AGE"]
            idx = df[df['AGE']==age].index.values.astype(int)[0]
            new_age = float("{:.2f}".format(age/12))
            df.loc[idx, "AGE"] = new_age
    elif file.split("\\")[1] == "Mwalimu.csv":
        df.loc[missing_age, "AGE"] = np.random.choice(range(18, 25), size=len(df[missing_age]))
   
    #df.to_csv(r"projects_csv/complete/complete_"+file.split("\\")[1])

# Cleaning the data
After qualitative judgement of some data it was noticed that some names were reported differently from CSV to CSV and so a cleaning was required. <br>
Projects that reported participants, outside villages described by the association were restricted to Mi.Fi.Ma. and the small amount of paticipants (less than 15) residing outside Pomerini were registered as Pomerinians.

In [ ]:
to_scan= [file for file in get_all_in_dir("projects_csv/complete")]
for file in to_scan:
    df = pd.read_csv(file, encoding="utf-8")
    idx = df.index.values.astype(int)
    for el in idx:
        if df.loc[el, "VILLAGE"] == "Kihesa Mgagao":
            df.loc[el, "VILLAGE"] = "Khesa Mgagao"
        if df.loc[el, "VILLAGE"] in ["Kaole", "Iringa", "Mbeya", "Lukani", "Ilula", "Ilula (Ikokoto)", "Image", "Dabaga", "Itungi"]:
            df.loc[el, "VILLAGE"] = "Pomerini"
    #df.to_csv(r"projects_csv/complete/"+file.split("\\")[1])

# Generating Villages complete files
In order to represent the brader villages environment, random graphs were generated in Gephi: this was also done basing on the information given from the association and corresponds to their averaging of the population.<br>
The villages under consideration were:
<ol>
<li><b>Khesa Mgagao</b>: ca. 2000 inhabitants </li>
<li><b>Kitowo</b>: ca. 3000 inhabitants </li>
<li><b>Mwawambala</b>: ca. 3000 inhabitants </li>
<li><b>Pomerini</b>: ca. 3000 inhabitants </li>
<li><b>Ukumbi</b>: ca. 3000 inhabitants </li>
</ol>

Basing on the information provided by the association, the oscillation in population and their unawareness of the real number of villagers, the number of inhabitants was set in a range of +- 250 around their averaging.

In [ ]:
rand_kitowo=random.randint(2750, 3250)
rand_ukumbi=random.randint(2750, 3250)
rand_mwawambala=random.randint(2750, 3250)
rand_pomerini=random.randint(2750, 3250)
rand_khesa_mgagao=random.randint(1750, 2250)

id=0

age_df=pd.read_csv("projects_csv/measures\\age_count.csv", encoding="utf-8")
agetotal_df=pd.read_csv("projects_csv/measures\\agetotal_count.csv", encoding="utf-8")
age_total_df = pd.concat([age_df,agetotal_df],ignore_index=True)
idx = age_total_df.index.values.astype(int)
total_age = age_total_df['PEOPLE'].sum() 
for el in idx:
    age_total_df.loc[el, "PEOPLE"] = age_total_df.loc[el, "PEOPLE"]/total_age


village_dict = {"Kitowo": rand_kitowo, "Khesa Mgagao": rand_khesa_mgagao, "Mawambala": rand_mwawambala, "Pomerini": rand_pomerini, "Ukumbi": rand_ukumbi}

for village in village_dict.keys():
    df=pd.DataFrame(index=np.arange(village_dict[village]), columns=["ID","NAME", "PROJECT", "AGE", "SEX", "FAMILY", "START", "VILLAGE"])
    idx = df.index.values.astype(int)
    missing_age = df["AGE"].isnull()
    missing_family=df["FAMILY"].isnull()
    missing_sex=df["SEX"].isnull()
    missing_id = df["ID"].isnull()
    df.loc[missing_age,'AGE'] = np.random.choice(age_total_df["AGE"], size=len(df[missing_age]),p=age_total_df["PEOPLE"] )
    df.loc[missing_sex, "SEX"] = np.random.choice(["M", "F"], size=len(df[missing_sex]))
    df.loc[missing_family, "FAMILY"] = np.random.choice(df_family["FAMILY"], size=len(df[missing_family]), p=df_family["PERCENTAGE"])
    for el in idx[1:]:
        df.loc[el, "NAME"] = "Name"+str(el)
        df.loc[el, "ID"] = id+el
        df.loc[el, "VILLAGE"] = village
    df=df.dropna(thresh=4)
    id=id+village_dict[village]-1
    
    #df.to_csv(r"projects_csv/complete/complete_"+village+".csv")


Now that we have the villages networks we can simply extract the data from the projects CSVs and substitute random generated rows with the ones belonging to projects set in that village. <br> 
In this way we will obtain a description of the situation for each village but, since what we want is a wider view on the project's influence on the community not only in villages but as a cross bridge between them, we ought to generate a complete network of Tulims's influence in the Pomerini zone.

In [ ]:
to_scan= [file for file in get_all_in_dir("projects_csv/complete")]
villages = list()
projects = list()
for file in to_scan:
    if file.split("/")[1].split("\\")[1].split("_")[1].split(".")[0] in ["Pomerini", "Mawambala", "Kitowo", "Khesa Mgagao", "Ukumbi"]:
        villages.append(file)
    else:
        projects.append(file)

idx_dict = {"Kitowo": 1, "Khesa Mgagao": 1, "Mawambala": 1, "Pomerini": 1, "Ukumbi": 1}

for project in projects:
    df_project = pd.read_csv(project, encoding="utf-8", index_col="Unnamed: 0")
    project_name = project.split("/")[1].split("\\")[1].split("_")[1].split(".")[0]
    idx = df_project.index.values.astype(int)
    for el in idx:
        village = df_project.loc[el, "VILLAGE"]
        try:
            df_village = pd.read_csv("projects_csv/complete/complete_"+village+".csv", encoding="utf-8", index_col="Unnamed: 0")
            df_village.loc[idx_dict[village], "NAME"] = df_project.loc[el, "NAME"]
            df_village.loc[idx_dict[village], "VILLAGE"] = df_project.loc[el, "VILLAGE"]
            df_village.loc[idx_dict[village], "PROJECT"] = project_name
            df_village.loc[idx_dict[village], "AGE"] = df_project.loc[el, "AGE"]
            df_village.loc[idx_dict[village], "SEX"] = df_project.loc[el, "SEX"]
            df_village.loc[idx_dict[village], "FAMILY"] = df_project.loc[el, "FAMILY"]
            df_village.loc[idx_dict[village], "START"] = df_project.loc[el, "START"]
            #df_village.to_csv(r"projects_csv/complete/complete_"+village+".csv")
            idx_dict[village]+=1
        except Exception as e:
            print (e)

    


Now we can mesh up those networks and create a wider one containing all information regarding projects.

In [ ]:
to_scan= [file for file in get_all_in_dir("projects_csv/complete")]
villages = list()
for file in to_scan:
    if file.split("/")[1].split("\\")[1].split("_")[1].split(".")[0] in ["Pomerini", "Mawambala", "Kitowo", "Khesa Mgagao", "Ukumbi"]:
        villages.append(file)

df_total=pd.DataFrame(columns=["ID", "NAME", "PROJECT", "AGE", "SEX", "FAMILY", "START", "VILLAGE"])

for village in villages:
    village_name = village.split("/")[1].split("\\")[1].split("_")[1].split(".")[0]
    df = pd.read_csv(village, encoding="utf-8", index_col="Unnamed: 0")
    df_total = pd.concat([df_total, df], ignore_index=True)
    missing_village=df_total["VILLAGE"].isnull()
    df_total.loc[missing_village, "VILLAGE"] = village_name

#df_total.to_csv(r"projects_csv/complete/whole_network.csv")

Now we can manually migrate files in a dedicated folder, namely "villages" and set the id as index.

In [ ]:
to_scan= [file for file in get_all_in_dir("projects_csv/villages")]
for file in to_scan:
    df = pd.read_csv(file, encoding="utf-8")
    df = df.drop("Unnamed: 0", axis=1)
    df=df.set_index("ID")
    #df.to_csv(file)

# Creating village networks
Now, in order to create our network we need to set rules in order to discretely decide if two nodes are connected. <br>
The main discriminants would be: 
<ol>
<li><b>Being part of the same project</b>: this should be the most valuable kind of connection, of weight 3, since what we are trying to investigate are the connections created by the projects; </li>
<li><b>Being part of the same family</b>: this kind of connection has definitely a heavier weight, since people living in the same family core should exchange information and positive or negative reinforcements that can weight on project's efficience; </li>
<li><b>Being part of the same village</b></li>
</ol>

So now every node should present:
 <ol>
<li>A number of connection of <b>weight 2</b> equal to the <b>number of participants in its project minus one</b> (itself)</li>
<li>A number of connection of <b>weight 1</b> equal to the <b>number of people in its family core</b></li>
</ol>

Also in this phase it was possible to balance incoherences in family components distribution: simply after a first cycle of edges is created it is possible that some names are still present in the starting dataset.<br>
This is because it is very difficult for families to be distributed so that at the end every component is in a core: the problem was solved by grouping leftovers as if they were a family core on their own of size equal to their quantity. Anyway it is impossible that this quantity exceeds the maximum amount of people per family, because otherway some of them would group together and lower again the quantity below 8, which is the maximum width of family cores.<br>
This new family core is not only included as a set of edges but substituted to the previous in the original dataset.

In [ ]:
to_scan= [file for file in get_all_in_dir("projects_csv/villages")]

for file in to_scan:
    df = pd.read_csv(file, encoding="utf-8")
    df = df.sample(frac=1, ignore_index=True)

    df_network = pd.DataFrame(columns=["SOURCE", "TARGET", "WEIGHT"])
    df_node = pd.DataFrame(columns=["SOURCE", "TARGET", "WEIGHT"])

    if file != "projects_csv/villages\\whole_network.csv":
        file_name = file.split("/")[1].split("\\")[1].split("_")[1].split(".")[0]
        idx = df.index.values.astype(int)
        name_list = list()
        for el in idx:
            try:
                n=1
                for el_2 in idx:
                    if n<=int(df.loc[el, "FAMILY"]):
                        if df.loc[el, "FAMILY"] == df.loc[el_2, "FAMILY"] and df.loc[el, "ID"] != df.loc[el_2, "ID"]:
                            df_node.loc[1,"SOURCE"] = df.loc[el, "ID"]
                            df_node.loc[1,"TARGET"] = df.loc[el_2, "ID"]
                            df_node.loc[1,"WEIGHT"] = 1
                            df_network = pd.concat([df_network, df_node],ignore_index=True)
                            family_count = int(df.loc[el, "FAMILY"])
                            n+=1
                            df = df.drop(el_2)
                            idx = df.index.values.astype(int)
                        else:
                            continue     
                    else:
                        idx_last_edge = len(df_network)-1
                        for i in range(family_count):
                            target_count=family_count-(i+1)
                            c=1
                            for z in range(target_count):
                                pointer_source = idx_last_edge-i
                                pointer = pointer_source-c
                                df_node.loc[1,"SOURCE"] = df_network.loc[pointer_source, "TARGET"]
                                df_node.loc[1,"TARGET"] = df_network.loc[pointer, "TARGET"]
                                df_node.loc[1,"WEIGHT"] = 1
                                df_network = pd.concat([df_network, df_node], ignore_index=True)
                                c+=1
                        df = df.drop(el)
                        idx = df.index.values.astype(int)
                        break
            except Exception as e:
                continue
        

        if len(df)>0:
            idx = df.index.values.astype(int)
            for el in idx[:1]:
                name_list.append(df.loc[el, "ID"])
                for el_2 in idx[1:]:
                    df_node.loc[1,"SOURCE"] = df.loc[el, "ID"]
                    df_node.loc[1,"TARGET"] = df.loc[el_2, "ID"]
                    df_node.loc[1,"WEIGHT"] = 1
                    df_network = pd.concat([df_network, df_node], ignore_index = True)
                    family_count = int(df.loc[el, "FAMILY"])
                    name_list.append(df.loc[el_2, "ID"])
                    df = df.drop(el_2)
            idx_last_edge = len(df_network)-1
            for i in range(family_count):
                target_count=family_count-(i+1)
                c=1
                for z in range(target_count):
                    pointer_source = idx_last_edge-i
                    pointer = pointer_source-c
                    df_node.loc[1,"SOURCE"] = df_network.loc[pointer_source, "TARGET"]
                    df_node.loc[1,"TARGET"] = df_network.loc[pointer, "TARGET"]
                    df_node.loc[1,"WEIGHT"] = 1
                    df_network = pd.concat([df_network, df_node], ignore_index=True)
                    c+=1
            df = df.drop(el)
        

        df = pd.read_csv(file, encoding="utf-8")
        df_whole = pd.read_csv("projects_csv/villages\\whole_network.csv", encoding="utf-8")

        idx = df.index.values.astype(int)
        for el in idx:
            if df.loc[el, "ID"] in name_list:
                name = df.loc[el, "ID"]
                df.loc[el, "FAMILY"] = len(name_list)
                idx_name = df_whole.index[(df_whole["ID"] == name) & (df_whole["VILLAGE"] == file_name)]
                df_whole.loc[idx_name, "FAMILY"] = len(name_list)
        
        #df.to_csv(r"projects_csv/villages/complete_"+file_name+".csv")

        df_whole = df_whole.set_index('ID')
        #df_whole.to_csv(r"projects_csv/villages\\whole_network.csv")

        idx = df.index.values.astype(int)
        for el in idx:
            try:
                project_count = 0
                if df.loc[el, "PROJECT"] != "Maji":
                    for el_2 in idx[el:]:
                        if df.loc[el, "PROJECT"] == df.loc[el_2, "PROJECT"] and df.loc[el, "ID"] != df.loc[el_2, "ID"]:
                            df_node.loc[1,"SOURCE"] = df.loc[el, "ID"]
                            df_node.loc[1,"TARGET"] = df.loc[el_2, "ID"]
                            df_node.loc[1,"WEIGHT"] = 2
                            df_network = pd.concat([df_network, df_node],ignore_index=True)
                            project_count +=1
                            df = df.drop(el_2) 
                        else:
                            continue        
                    idx_last_edge = len(df_network)-1
                    for i in range(project_count):
                        target_count=project_count-(i+1)
                        c=1
                        for z in range(target_count):
                            pointer_source = idx_last_edge-i
                            pointer = pointer_source-c
                            df_node.loc[1,"SOURCE"] = df_network.loc[pointer_source, "TARGET"]
                            df_node.loc[1,"TARGET"] = df_network.loc[pointer, "TARGET"]
                            df_node.loc[1,"WEIGHT"] = 2
                            df_network = pd.concat([df_network, df_node], ignore_index=True)
                            c+=1
                    df = df.drop(el)
            except Exception as e:
                continue        

        #df_network.to_csv(r"projects_csv/networks/"+file_name+"_edges.csv")


Now we can create a file containing edges of the whole network

In [ ]:
to_scan= [file for file in get_all_in_dir("projects_csv/networks")]
df_whole_edges = pd.DataFrame(columns=["SOURCE", "TARGET", "WEIGHT"])
for file in to_scan:
    df = pd.read_csv(file, encoding="utf-8")
    df_whole_edges = pd.concat([df_whole_edges, df], ignore_index=True)

df=pd.read_csv("projects_csv/villages\\whole_network.csv", encoding="utf-8")
idx = df.index.values.astype(int)
for el in idx:
    try:
        project_count = 0
        if df.loc[el, "PROJECT"] != "Maji":
            for el_2 in idx[el:]:
                if df.loc[el, "PROJECT"] == df.loc[el_2, "PROJECT"] and df.loc[el, "ID"] != df.loc[el_2, "ID"] and df.loc[el, "VILLAGE"] != df.loc[el_2, "VILLAGE"]:
                    df_node.loc[1,"SOURCE"] = df.loc[el, "ID"]
                    df_node.loc[1,"TARGET"] = df.loc[el_2, "ID"]
                    df_node.loc[1,"WEIGHT"] = 2
                    df_whole_edges = pd.concat([df_whole_edges, df_node],ignore_index=True)
                    project_count +=1
                    df = df.drop(el_2) 
                else:
                    continue        
            idx_last_edge = len(df_whole_edges)-1
            for i in range(project_count):
                target_count=project_count-(i+1)
                c=1
                for z in range(target_count):
                    pointer_source = idx_last_edge-i
                    pointer = pointer_source-c
                    df_node.loc[1,"SOURCE"] = df_whole_edges.loc[pointer_source, "TARGET"]
                    df_node.loc[1,"TARGET"] = df_whole_edges.loc[pointer, "TARGET"]
                    df_node.loc[1,"WEIGHT"] = 2
                    df_whole_edges = pd.concat([df_whole_edges, df_node], ignore_index=True)
                    c+=1
            df = df.drop(el)
    except Exception as e:
        continue

df_whole_edges = df_whole_edges.drop_duplicates(keep='last')
#df_whole_edges.to_csv(r"projects_csv/networks/whole_edges.csv")              


The last thing left to do is clean the data as to process it in Gephi, by removing empty cells. <br>

In [ ]:
to_scan= [file for file in get_all_in_dir("projects_csv/villages")]
for file in to_scan:
    file_name = file.split("/")[1].split("\\")[1].split("_")[1].split(".")[0]
    df = pd.read_csv(file, encoding="utf-8")
    df = df.replace(r'^\s*$', np.nan, regex=True)
    try:
        df= df.drop("Unnamed: 0", axis=1)
    except Exception:
        df[["AGE", "FAMILY", "START"]] = df[["AGE", "FAMILY", "START"]].apply(pd.to_numeric)
        df.to_csv(r"projects_csv/networks/"+file_name+"_nodes.csv")
    df[["AGE", "FAMILY", "START"]] = df[["AGE", "FAMILY", "START"]].apply(pd.to_numeric)
    #df.to_csv(r"projects_csv/networks/"+file_name+"_nodes.csv")
        

Now we can index the new files and the nwtworks are complete.

In [ ]:
to_scan= [file for file in get_all_in_dir("projects_csv/networks")]
for file in to_scan:
    file_name = file.split("/")[1].split("\\")[1].split("_")[1].split(".")[0]
    df = pd.read_csv(file, encoding="utf-8")
    if file_name == "edges": 
        df = df.set_index("SOURCE")
    else:
        df = df.set_index("ID")
    try: 
        df = df.drop("Unnamed: 0", axis=1)
        #df.to_csv(file)
    except Exception:
        #df.to_csv(file)

# Add data regarding friends and working relatioship
 
Another possible approach is to use literature on this topic to extract data regarding significant communities such as friendship and co-working. <br>

In "The role of strong-tie social networks in mediating food security of fish
resources by a traditional riverine community in the Brazilian Amazon" (2015, F. Merthes et al.) communities based on friendship have been find to fall in a range from 2 to 29 nodes connected; also work communities fall in a similar range, from 2 to 30. In the study it was found that, upon 915 that came down to 793 when considering overlappings, relationships, 177 were of friendship kind, 167 of working type and 571 based on kinship.<br>

From this data, and the distribution of kinship relationship we have at disposal, we can easily recreate a model that follows the distribution evidentiated in this study and fills the gaps created by missing edge weights. This model should present nodes with edges strctured as follows:
 <ol>
<li>A number of connection of <b>weight 2</b> equal to the <b>number of participants in its project minus one</b> (itself)</li>
<li>A number of connection of <b>weight 1</b> equal to the <b>number of people in its family core</b> plus the <b>number of people in its friends and co-working community minus one</b> (itself)</li>
</ol> 

Some small corrections were carried on in random assignation of value: irst of all children do not work, so their value of relationship through work was set to 0 and second no one has no friends so, in case available ties ended a random value between 1 and 3 was extracted.

In [72]:
to_scan = [file for file in get_all_in_dir("projects_csv/villages")]

for file in to_scan:
    if file != "projects_csv/villages\\whole_network.csv":
        file_name = file.split("/")[1].split("\\")[1].split("_")[1].split(".")[0]
        df = pd.read_csv("projects_csv/networks/"+file_name+"_edges.csv", encoding="utf-8")
        conversion_factor = df["WEIGHT"].value_counts()[1]/571
        friendship_ties = np.floor(177*conversion_factor)
        working_ties = np.floor(167*conversion_factor)

        df = pd.read_csv(file, encoding="utf-8", index_col="Unnamed: 0")
        df = df.sample(frac=1, ignore_index=True)
        df["FRIENDS"] = np.nan
        df["WORK"] = np.nan
        df_nowork = df.index[(df['PROJECT'] == "TupoPamoja") | (df['PROJECT'] == "PiantalaSubito")]
        df.loc[df_nowork,'WORK'] = 0
        missing_friends = df["FRIENDS"].isnull()
        df.loc[missing_friends,'FRIENDS'] = np.random.choice(range(2, 29), size=len(df[missing_friends]))
        missing_work = df["WORK"].isnull()
        df.loc[missing_work,'WORK'] = np.random.choice(range(2, 30), size=len(df[missing_work]))

        df_network = pd.DataFrame(columns=["SOURCE", "TARGET", "WEIGHT"])
        df_node = pd.DataFrame(columns=["SOURCE", "TARGET", "WEIGHT"])
        df_nodes = pd.DataFrame(columns = ["ID", "NAME", "PROJECT", "AGE", "SEX", "FAMILY", "START", "FRIENDS", "WORK"])
        to_add = pd.DataFrame(columns = ["ID", "NAME", "PROJECT", "AGE", "SEX", "FAMILY", "START", "FRIENDS", "WORK"])

        file_name = file.split("/")[1].split("\\")[1].split("_")[1].split(".")[0]
        idx = df.index.values.astype(int)
        for el in idx:
            try:
                n=1
                for el_2 in idx: 
                    if n<=int(df.loc[el, "FRIENDS"]):
                        if df.loc[el, "FRIENDS"] == df.loc[el_2, "FRIENDS"] and df.loc[el, "ID"] != df.loc[el_2, "ID"] and friendship_ties>0:
                            to_add.loc[1, "ID"] = df.loc[el_2, "ID"]
                            to_add.loc[1, "NAME"] = df.loc[el_2, "NAME"]
                            to_add.loc[1, "PROJECT"] = df.loc[el_2, "PROJECT"]
                            to_add.loc[1, "AGE"] = df.loc[el_2, "AGE"]
                            to_add.loc[1, "SEX"] = df.loc[el_2, "SEX"]
                            to_add.loc[1, "FAMILY"] = df.loc[el_2, "FAMILY"]
                            to_add.loc[1, "START"] = df.loc[el_2, "START"]
                            to_add.loc[1, "FRIENDS"] = df.loc[el_2, "FRIENDS"]
                            to_add.loc[1, "WORK"] = df.loc[el_2, "WORK"]
                            df_nodes = pd.concat([df_nodes, to_add],ignore_index=True)
                            df_node.loc[1,"SOURCE"] = df.loc[el, "ID"]
                            df_node.loc[1,"TARGET"] = df.loc[el_2, "ID"]
                            df_node.loc[1,"WEIGHT"] = 1
                            df_network = pd.concat([df_network, df_node],ignore_index=True)
                            friends_count = int(df.loc[el, "FRIENDS"])
                            n+=1
                            df = df.drop(el_2)
                            idx = df.index.values.astype(int)
                        elif friendship_ties<=0:
                            to_add.loc[1, "ID"] = df.loc[el_2, "ID"]
                            to_add.loc[1, "NAME"] = df.loc[el_2, "NAME"]
                            to_add.loc[1, "PROJECT"] = df.loc[el_2, "PROJECT"]
                            to_add.loc[1, "AGE"] = df.loc[el_2, "AGE"]
                            to_add.loc[1, "SEX"] = df.loc[el_2, "SEX"]
                            to_add.loc[1, "FAMILY"] = df.loc[el_2, "FAMILY"]
                            to_add.loc[1, "START"] = df.loc[el_2, "START"]
                            to_add.loc[1, "FRIENDS"] = random.randint(1,3)
                            to_add.loc[1, "WORK"] = df.loc[el_2, "WORK"]
                            df_nodes = pd.concat([df_nodes, to_add],ignore_index=True)
                            break     
                    elif friendship_ties>0:
                        to_add.loc[1, "ID"] = df.loc[el, "ID"]
                        to_add.loc[1, "NAME"] = df.loc[el, "NAME"]
                        to_add.loc[1, "PROJECT"] = df.loc[el, "PROJECT"]
                        to_add.loc[1, "AGE"] = df.loc[el, "AGE"]
                        to_add.loc[1, "SEX"] = df.loc[el, "SEX"]
                        to_add.loc[1, "FAMILY"] = df.loc[el, "FAMILY"]
                        to_add.loc[1, "START"] = df.loc[el, "START"]
                        to_add.loc[1, "FRIENDS"] = df.loc[el, "FRIENDS"]
                        to_add.loc[1, "WORK"] = df.loc[el, "WORK"]
                        df_nodes = pd.concat([df_nodes, to_add],ignore_index=True)
                        idx_last_edge = len(df_network)-1
                        for i in range(friends_count):
                            friendship_ties = friendship_ties-(friends_count-i)
                            target_count=friends_count-(i+1)
                            c=1
                            for z in range(target_count):
                                pointer_source = idx_last_edge-i
                                pointer = pointer_source-c
                                df_node.loc[1,"SOURCE"] = df_network.loc[pointer_source, "TARGET"]
                                df_node.loc[1,"TARGET"] = df_network.loc[pointer, "TARGET"]
                                df_node.loc[1,"WEIGHT"] = 1
                                df_network = pd.concat([df_network, df_node], ignore_index=True)
                                c+=1
                        df = df.drop(el)
                        idx = df.index.values.astype(int)
                        break
                    else:
                        to_add.loc[1, "ID"] = df.loc[el, "ID"]
                        to_add.loc[1, "NAME"] = df.loc[el, "NAME"]
                        to_add.loc[1, "PROJECT"] = df.loc[el, "PROJECT"]
                        to_add.loc[1, "AGE"] = df.loc[el, "AGE"]
                        to_add.loc[1, "SEX"] = df.loc[el, "SEX"]
                        to_add.loc[1, "FAMILY"] = df.loc[el, "FAMILY"]
                        to_add.loc[1, "START"] = df.loc[el, "START"]
                        to_add.loc[1, "FRIENDS"] = random.randint(1,3)
                        to_add.loc[1, "WORK"] = df.loc[el, "WORK"]
                        df_nodes = pd.concat([df_nodes, to_add],ignore_index=True)
                        break
            except Exception as e:
                continue

        if len(df)>0 and friendship_ties>0:
            idx = df.index.values.astype(int)
            for el in idx:
                try:
                    n=1
                    for el_2 in idx: 
                        if n<=len(df):
                            if df.loc[el, "FRIENDS"] == df.loc[el_2, "FRIENDS"] and df.loc[el, "ID"] != df.loc[el_2, "ID"] and working_ties>0:
                                to_add.loc[1, "ID"] = df.loc[el_2, "ID"]
                                to_add.loc[1, "NAME"] = df.loc[el_2, "NAME"]
                                to_add.loc[1, "PROJECT"] = df.loc[el_2, "PROJECT"]
                                to_add.loc[1, "AGE"] = df.loc[el_2, "AGE"]
                                to_add.loc[1, "SEX"] = df.loc[el_2, "SEX"]
                                to_add.loc[1, "FAMILY"] = df.loc[el_2, "FAMILY"]
                                to_add.loc[1, "START"] = df.loc[el_2, "START"]
                                to_add.loc[1, "FRIENDS"] = df.loc[el_2, "FRIENDS"]
                                to_add.loc[1, "WORK"] = df.loc[el_2, "WORK"]
                                df_nodes = pd.concat([df_nodes, to_add],ignore_index=True)
                                df_node.loc[1,"SOURCE"] = df.loc[el, "ID"]
                                df_node.loc[1,"TARGET"] = df.loc[el_2, "ID"]
                                df_node.loc[1,"WEIGHT"] = 1
                                df_network = pd.concat([df_network, df_node],ignore_index=True)
                                friends_count = int(df.loc[el, "FRIENDS"])
                                n+=1
                                df = df.drop(el_2)
                                idx = df.index.values.astype(int)
                            elif friendship_ties<=0:
                                to_add.loc[1, "ID"] = df.loc[el_2, "ID"]
                                to_add.loc[1, "NAME"] = df.loc[el_2, "NAME"]
                                to_add.loc[1, "PROJECT"] = df.loc[el_2, "PROJECT"]
                                to_add.loc[1, "AGE"] = df.loc[el_2, "AGE"]
                                to_add.loc[1, "SEX"] = df.loc[el_2, "SEX"]
                                to_add.loc[1, "FAMILY"] = df.loc[el_2, "FAMILY"]
                                to_add.loc[1, "START"] = df.loc[el_2, "START"]
                                to_add.loc[1, "FRIENDS"] = random.randint(1,3)
                                to_add.loc[1, "WORK"] = df.loc[el_2, "WORK"]
                                df_nodes = pd.concat([df_nodes, to_add],ignore_index=True)
                                break     
                        elif friendship_ties>0:
                            to_add.loc[1, "ID"] = df.loc[el, "ID"]
                            to_add.loc[1, "NAME"] = df.loc[el, "NAME"]
                            to_add.loc[1, "PROJECT"] = df.loc[el, "PROJECT"]
                            to_add.loc[1, "AGE"] = df.loc[el, "AGE"]
                            to_add.loc[1, "SEX"] = df.loc[el, "SEX"]
                            to_add.loc[1, "FAMILY"] = df.loc[el, "FAMILY"]
                            to_add.loc[1, "START"] = df.loc[el, "START"]
                            to_add.loc[1, "FRIENDS"] = df.loc[el, "FRIENDS"]
                            to_add.loc[1, "WORK"] = df.loc[el, "WORK"]
                            df_nodes = pd.concat([df_nodes, to_add],ignore_index=True)
                            idx_last_edge = len(df_network)-1
                            for i in range(friends_count):
                                friendship_ties = friendship_ties-(friends_count-i)
                                target_count=friends_count-(i+1)
                                c=1
                                for z in range(target_count):
                                    pointer_source = idx_last_edge-i
                                    pointer = pointer_source-c
                                    df_node.loc[1,"SOURCE"] = df_network.loc[pointer_source, "TARGET"]
                                    df_node.loc[1,"TARGET"] = df_network.loc[pointer, "TARGET"]
                                    df_node.loc[1,"WEIGHT"] = 1
                                    df_network = pd.concat([df_network, df_node], ignore_index=True)
                                    c+=1
                            df = df.drop(el)
                            idx = df.index.values.astype(int)
                            break
                        else:
                            break
                except Exception as e:
                    continue
        elif len(df)>0:
            idx = df.index.values.astype(int)
            for el in idx:
                to_add.loc[1, "ID"] = df.loc[el, "ID"]
                to_add.loc[1, "NAME"] = df.loc[el, "NAME"]
                to_add.loc[1, "PROJECT"] = df.loc[el, "PROJECT"]
                to_add.loc[1, "AGE"] = df.loc[el, "AGE"]
                to_add.loc[1, "SEX"] = df.loc[el, "SEX"]
                to_add.loc[1, "FAMILY"] = df.loc[el, "FAMILY"]
                to_add.loc[1, "START"] = df.loc[el, "START"]
                to_add.loc[1, "FRIENDS"] = random.randint(1,3)
                to_add.loc[1, "WORK"] = df.loc[el, "WORK"]
                df_nodes = pd.concat([df_nodes, to_add],ignore_index=True)
                df = df.drop(el)

        #filling work

        df = df_nodes       
        df = df.sample(frac=1, ignore_index=True)

        file_name = file.split("/")[1].split("\\")[1].split("_")[1].split(".")[0]
        idx = df.index.values.astype(int)
        for el in idx:
            try:
                n=1
                for el_2 in idx:
                    if n<=int(df.loc[el, "WORK"]): 
                        if df.loc[el, "WORK"] == df.loc[el_2, "WORK"] and df.loc[el, "ID"] != df.loc[el_2, "ID"] and working_ties>0:
                            to_add.loc[1, "ID"] = df.loc[el_2, "ID"]
                            to_add.loc[1, "NAME"] = df.loc[el_2, "NAME"]
                            to_add.loc[1, "PROJECT"] = df.loc[el_2, "PROJECT"]
                            to_add.loc[1, "AGE"] = df.loc[el_2, "AGE"]
                            to_add.loc[1, "SEX"] = df.loc[el_2, "SEX"]
                            to_add.loc[1, "FAMILY"] = df.loc[el_2, "FAMILY"]
                            to_add.loc[1, "START"] = df.loc[el_2, "START"]
                            to_add.loc[1, "FRIENDS"] = df.loc[el_2, "FRIENDS"]
                            to_add.loc[1, "WORK"] = df.loc[el_2, "WORK"]
                            df_nodes = pd.concat([df_nodes, to_add], ignore_index=True)
                            df_node.loc[1,"SOURCE"] = df.loc[el, "ID"]
                            df_node.loc[1,"TARGET"] = df.loc[el_2, "ID"]
                            df_node.loc[1,"WEIGHT"] = 1
                            df_network = pd.concat([df_network, df_node],ignore_index=True)
                            work_count = int(df.loc[el, "WORK"])
                            n+=1
                            df = df.drop(el_2)
                            idx = df.index.values.astype(int)
                        elif working_ties<=0:
                            id_to_look = df.loc[el_2, "ID"]
                            df_nodes.loc[id_to_look, "WORK"] = 0  
                            break
                    elif working_ties>0:
                        idx_last_edge = len(df_network)-1
                        for i in range(work_count):
                            working_ties = working_ties-(work_count-i)
                            target_count=work_count-(i+1)
                            c=1
                            for z in range(target_count):
                                pointer_source = idx_last_edge-i
                                pointer = pointer_source-c
                                df_node.loc[1,"SOURCE"] = df_network.loc[pointer_source, "TARGET"]
                                df_node.loc[1,"TARGET"] = df_network.loc[pointer, "TARGET"]
                                df_node.loc[1,"WEIGHT"] = 1
                                df_network = pd.concat([df_network, df_node], ignore_index=True)
                                c+=1
                        df = df.drop(el)
                        idx = df.index.values.astype(int)
                        break
                    else:
                        break
            except Exception as e:
                continue
        

        if len(df)>0 and working_ties>0:
            idx = df.index.values.astype(int)
            for el in idx:
                try:
                    n=1
                    for el_2 in idx: 
                        if n<=len(df):
                            if df.loc[el, "WORK"] == df.loc[el_2, "WORK"] and df.loc[el, "ID"] != df.loc[el_2, "ID"] and working_ties>0:
                                to_add.loc[1, "ID"] = df.loc[el_2, "ID"]
                                to_add.loc[1, "NAME"] = df.loc[el_2, "NAME"]
                                to_add.loc[1, "PROJECT"] = df.loc[el_2, "PROJECT"]
                                to_add.loc[1, "AGE"] = df.loc[el_2, "AGE"]
                                to_add.loc[1, "SEX"] = df.loc[el_2, "SEX"]
                                to_add.loc[1, "FAMILY"] = df.loc[el_2, "FAMILY"]
                                to_add.loc[1, "START"] = df.loc[el_2, "START"]
                                to_add.loc[1, "FRIENDS"] = df.loc[el_2, "FRIENDS"]
                                to_add.loc[1, "WORK"] = df.loc[el_2, "WORK"]
                                df_nodes = pd.concat([df_nodes, to_add],ignore_index=True)
                                df_node.loc[1,"SOURCE"] = df.loc[el, "ID"]
                                df_node.loc[1,"TARGET"] = df.loc[el_2, "ID"]
                                df_node.loc[1,"WEIGHT"] = 1
                                df_network = pd.concat([df_network, df_node],ignore_index=True)
                                work_count = int(df.loc[el, "FRIENDS"])
                                n+=1
                                df = df.drop(el_2)
                                idx = df.index.values.astype(int)
                            elif working_ties<0:
                                id_to_look = df.loc[el_2, "ID"]
                                df_nodes.loc[id_to_look, "WORK"] = 0
                                break     
                        elif working_ties>0:
                            to_add.loc[1, "ID"] = df.loc[el, "ID"]
                            to_add.loc[1, "NAME"] = df.loc[el, "NAME"]
                            to_add.loc[1, "PROJECT"] = df.loc[el, "PROJECT"]
                            to_add.loc[1, "AGE"] = df.loc[el, "AGE"]
                            to_add.loc[1, "SEX"] = df.loc[el, "SEX"]
                            to_add.loc[1, "FAMILY"] = df.loc[el, "FAMILY"]
                            to_add.loc[1, "START"] = df.loc[el, "START"]
                            to_add.loc[1, "FRIENDS"] = df.loc[el, "FRIENDS"]
                            to_add.loc[1, "WORK"] = df.loc[el, "WORK"]
                            df_nodes = pd.concat([df_nodes, to_add],ignore_index=True)
                            idx_last_edge = len(df_network)-1
                            for i in range(work_count):
                                working_ties = working_ties-(work_count-i)
                                target_count=work_count-(i+1)
                                c=1
                                for z in range(target_count):
                                    pointer_source = idx_last_edge-i
                                    pointer = pointer_source-c
                                    df_node.loc[1,"SOURCE"] = df_network.loc[pointer_source, "TARGET"]
                                    df_node.loc[1,"TARGET"] = df_network.loc[pointer, "TARGET"]
                                    df_node.loc[1,"WEIGHT"] = 1
                                    df_network = pd.concat([df_network, df_node], ignore_index=True)
                                    c+=1
                            df = df.drop(el)
                            idx = df.index.values.astype(int)
                            break
                        else:
                            break
                except Exception as e:
                    continue
        elif len(df)>0:
            idx = df.index.values.astype(int)
            for el in idx:
                id_to_look = df.loc[el, "ID"]
                df_nodes.loc[id_to_look, "WORK"] = 0

            

        df_nodes = df_nodes.drop_duplicates(keep="last")
        df_nodes = df_nodes.set_index("ID")
        df_nodes.to_csv(r"projects_csv/villages_comparison/nodes_"+file_name+".csv")

        df_to_concat = pd.read_csv("projects_csv/networks\\"+ file_name+"_edges.csv", encoding="utf-8")
        df_network = pd.concat([df_network, df_to_concat], ignore_index=True)
        #df_network = df_network.drop("Unnamed: 0")
        df_network = df_network.set_index("SOURCE")
        
        df_network.to_csv(r"projects_csv/villages_comparison/edges_"+file_name+".csv")

        

KeyboardInterrupt: 

# Data cleaning
Now we can close up with the data cleaning: setting data type to columns and substituting blank values with spaces to make files readable from Gephi.

In [70]:
to_clean = [file for file in get_all_in_dir("projects_csv/villages_comparison")]
for file in to_clean:
    df = pd.read_csv(file, encoding="utf-8") 
    for col in df.columns:
        if col == "ID" or col == "AGE":
            df[col] =  df[col].astype(int)
        else:
            df = df.astype({col: str})
    df.to_csv(file)
           

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

Now we can proceed to create the whole network of villages.

In [54]:
df_whole = pd.read_csv("projects_csv/networks\\whole_edges.csv", encoding="utf-8")
df_whole_nodes = pd.DataFrame(columns = ["ID", "NAME", "PROJECT", "AGE", "SEX", "FAMILY", "START", "FRIENDS", "WORK"])
to_append = [file for file in get_all_in_dir("projects_csv/villages_comparison")]
for file in to_append:
    file_name = file.split("/")[1].split("\\")[1].split("_")[1].split(".")[0]
    if file_name == "edges":
        df_to_append = pd.read_csv(file, encoding="utf-8")
        df_whole = pd.concat([df_whole,df_to_append], ignore_index=True)
    else:
        df_to_append = pd.read_csv(file, encoding="utf-8")
        df_whole_nodes = pd.concat([df_whole_nodes, df_to_append], ignore_index=True)

df_whole.to_csv(r"projects_csv/villages_comparison/network_edges.csv")
df_whole_nodes.to_csv(r"projects_csv/villages_comparison/network_nodes.csv")

C:\Users\Giorgia Sampo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (2,5,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
